<a href="https://colab.research.google.com/github/Roytjoo/schiphol-assignment/blob/master/Schiphol_Assigment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

To create a demoable code I have chosen to use Google Codelab to show the code and a way to easily interact with Pyspark. For productionized code I definitly choose for Github and python packages and modules with in combination with Docker and Pycharm as IDE.

In [ ]:
# download and installation of PySpark, and installation of the packages findspark and itables
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q -N https://apache.newfountain.nl/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark itables pyspark_test

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [ ]:
# load extension for datatables
%load_ext google.colab.data_table

The google.colab.data_table extension is already loaded. To reload it, use:
  %reload_ext google.colab.data_table


In [ ]:
# import os + setting variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

In [ ]:
# import pyspark libraries and initialize Spark
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql.dataframe import DataFrame 
from pyspark.sql.window import Window
from pyspark.sql.functions import col, window, udf, current_timestamp, row_number, lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType
import time
from datetime import datetime
import logging
import urllib.request
import math
from itables import show
from google.colab import data_table
import pytest
from pyspark_test import assert_pyspark_df_equal

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
# set logging
log = logging.getLogger(__name__)
log.setLevel("INFO")

# Task 1

*Create a batch Spark job that read in the routes dataset. It should create an overview of the top 10 airports used as source airport. Write the output to a filesystem.*

I have split up the different parts into small function for maintainability reusability. I used docstrings to describe the functions and inserted logging for execution progress.

In [ ]:
# create a schema for the file 
route_schema = StructType([ \
    StructField("Airline", StringType(),True), \
    StructField("Airline ID", StringType(),True), \
    StructField("Source airport", StringType(),True), \
    StructField("Source airport ID", StringType(), True), \
    StructField("Destination airport", StringType(), True), \
    StructField("Destination airport ID", StringType(), True), \
    StructField("Codeshare", StringType(), True), \
    StructField("Stops", IntegerType(), True), \
    StructField("Equipment", StringType(), True)
])

In [ ]:
def extract_data(file_name: str, header: bool=False, schema: StructType=route_schema):
  """
  extracts data from a file by reading it into a spark datafarme
  @param file_name: name of the file 
  @param header: boolean if a header is used
  @param schema: the schema that should be used
  """
  log.info("extract data from file")

  df = spark.read \
        .load(file_name, format="csv", sep=",", header=header, schema=schema) 
    
  return df

In [ ]:
def extract_data_from_url(url: str, header: bool=False, schema: StructType=route_schema):
  """
  extracts data from a url, by downloading the file and extracting the data
  @param url: the url where teh file is located
  """
  log.info("extract data from url")

  file_name = os.path.split(url)[1]
  urllib.request.urlretrieve(url, file_name)
  
  df = extract_data(file_name, header, schema)
  
  return df

In [ ]:
def transform_data(df: DataFrame):
  """
  writes the data from a dataframe to a file
  transforms the data to get the top 10 airports that are used most often as source airport
  @param df: the input data
  """
  log.info("transform data")

  result = df.groupBy(col("Source airport")) \
      .count() \
      .orderBy(col("count").desc()) \
      .limit(10)
  
  return result

In [ ]:
def load_data(df: DataFrame, file: str, file_format: str="csv"):
  """
  @param df: the data
  @param file: the file name
  @param file_format: the format of the output file
  """
  log.info("load data to file")

  df.write.format(file_format).mode('overwrite') \
      .option("header", "true") \
      .save(file)

In [ ]:
url = "https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat"

##Results Task 1

In [ ]:
def task1():
  """
  synchronously execute the steps for the taks 1 job
  """
  log.info("start task 1")

  df = extract_data_from_url(url)
  result = transform_data(df)
  result.show()
  load_data(result, "output.csv", "csv")

  log.info("task 1 completed")

In [ ]:
task1()

INFO:__main__:start task 1
INFO:__main__:extract data from url
INFO:__main__:extract data from file
INFO:__main__:transform data
INFO:__main__:load data to file


+--------------+-----+
|Source airport|count|
+--------------+-----+
|           ATL|  915|
|           ORD|  558|
|           PEK|  535|
|           LHR|  527|
|           CDG|  524|
|           FRA|  497|
|           LAX|  492|
|           DFW|  469|
|           JFK|  456|
|           AMS|  453|
+--------------+-----+



INFO:__main__:task 1 completed


# Task 2 
*Use Spark structured streaming to change your job into a streaming job, and use the dataset file as a source.*

I used the RatestreamSource to create a stream of the routes with a timestamp. By joining those two together on row nmber I get a stream of routes with an event time.

In [ ]:
def get_stream(rowsPerSecond: int=1000):
  """
  Gets a dummy stream of x rows per second
  @param rowsPerSecond: number of rows per second that should be outputed
  """
  log.info("get stream")

  stream = spark \
    .readStream \
    .format("rate") \
    .option("rowsPerSecond", rowsPerSecond) \
    .option("numPartitions", "1") \
    .option("rampUpTime", "0") \
    .load()

  return stream

In [ ]:
def combine_to_stream():
  """
  combine the dummy stream with the routes source data to create a stream of routes
  """
  log.info("combine stream and source data")

  routes = extract_data_from_url(url)
  routes = routes \
            .withColumn('rand', lit("")) \
            .withColumn("row_num", row_number().over(Window.partitionBy().orderBy("rand")))

  stream = get_stream()

  return stream.join(routes, [(stream.value == routes.row_num)], how="inner")

## Results Task 2

In [ ]:
def task2():
  """
  Task 2 combine the data from the source file with time data to create a tream of data
  """
  log.info("start task 2")

  combine_to_stream()

  log.info("task 2 completed")

In [ ]:
task2()

INFO:__main__:start task 2
INFO:__main__:combine stream and source data
INFO:__main__:extract data from url
INFO:__main__:extract data from file
INFO:__main__:get stream
INFO:__main__:task 2 completed


# Task 3
*Next change your streaming job so the aggregations are done using sliding windows. Pick any window and sliding interval. The end result should be the top 10 airports used as source airport within each window. When choosing the window interval, keep the size of the dataset in mind.*

I used Pyspark Structure streaming, however this has some limitations with limit and order by, which can not easily be used in the streaming query. Therefore, I store the stream with the windows and bathc process the data in top 10's. With the function foreach_batch_function, each batch is stored after it is available, this function can easily be extended to store to multiple locations.

In [ ]:
def transform_data_window(df: DataFrame):
  """
  transforms the stremaing data to windowed data with counts for that window and source airport
  @param df: the input data
  """
  log.info("transform data to windowing data")

  result = df \
    .withColumn("timestamp", current_timestamp()) \
    .withWatermark("timestamp", "5 seconds") \
    .groupBy(window(col("timestamp"), "10 seconds"), col("Source airport")) \
    .count() \
    .select("window.start", "window.end", "Source airport", "count")    

  return result

In [ ]:
def foreach_batch_function(batch_df: DataFrame, batch_id: int, output_file_name: str):
  """
  process each batch and store the data to file, by overwriting the previous data
  in 1 partition, to easily check for a human
  @param batch_df: the data of the batch
  @param bath_id: the batch id
  @param output_file_name: teh name where the batch should be stored
  """
  log.info("write batch to file")

  batch_df \
    .repartition(1) \
    .write.mode("overwrite") \
    .option("header", "true") \
    .csv(output_file_name)

In [ ]:
def start_streaming_load(transformation: DataFrame, output_file_name: str, max_secs: int=100):
  """
  starts the streaming for a max secs and loads the windowed data to an file
  @param transformation: the transformation data
  @param output_file_name: output file name where the data should be stored
  @param max_secs: max seconds to run for this streaming demo
  """
  log.info(f"saving stream to file for max {max_secs} secs")

  query = transformation \
    .writeStream \
    .foreachBatch(lambda batch_df, batch_id: foreach_batch_function(batch_df, batch_id, output_file_name)) \
    .outputMode("complete") \
    .start()

  t = 0

  while t <= max_secs:
      sleep_secs = 5
      time.sleep(sleep_secs)
      t += sleep_secs

  query.stop()

In [ ]:
def transform_windowed_data(df: DataFrame):
  """
  transforms the window data into top 10's per window by partitioning
  @param df: the input data
  """
  log.info("transform data to top 10 per window")

  result = df.withColumn("window_rank", \
                          row_number().over(Window.partitionBy("start").orderBy(col("count").desc(), col("Source airport")))) \
      .filter(col("window_rank") <= 10) \
      .orderBy(col("start"), col("count").desc())
  
  return result

## Results Task 3

In [ ]:
def task3():
  """
  synchronously execute the steps for the taks 3 job by loading the stream to file
  and partition the data from the windows into top 10's, since the current PySpark
  Structure streaming does not allow order by and limits in the streaming query
  """
  log.info("start task 3")

  output_file_name = "stream_batch_all.csv"

  stream = combine_to_stream()
  transform = transform_data_window(stream)
  start_streaming_load(transform, output_file_name)
  
  result_schema = StructType([ \
    StructField("start", TimestampType(), True), \
    StructField("end", TimestampType(), True), \
    StructField("Source airport", StringType(),True), \
    StructField("count", IntegerType(), True)
  ])

  df = extract_data(output_file_name, header=True, schema=result_schema)
  df_top10 = transform_windowed_data(df)
  data_table.DataTable(df_top10.toPandas(), include_index=False, num_rows_per_page=10)

  log.info("task 3 completed")

In [ ]:
task3()

INFO:__main__:start task 3
INFO:__main__:combine stream and source data
INFO:__main__:extract data from url
INFO:__main__:extract data from file
INFO:__main__:get stream
INFO:__main__:transform data to windowing data
INFO:__main__:saving stream to file for max 100 secs
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:extract data from file
INFO:__main__:transform data to top 10 per window
INFO:__main__:task 3 completed


In [ ]:
output_file_name = "stream_batch_all.csv"

stream = combine_to_stream()
transform = transform_data_window(stream)
start_streaming_load(transform, output_file_name)

result_schema = StructType([ \
  StructField("start", TimestampType(), True), \
  StructField("end", TimestampType(), True), \
  StructField("Source airport", StringType(),True), \
  StructField("count", IntegerType(), True)
])

df = extract_data(output_file_name, header=True, schema=result_schema)
df_top10 = transform_windowed_data(df)
data_table.DataTable(df_top10.toPandas(), include_index=False, num_rows_per_page=10)

INFO:__main__:combine stream and source data
INFO:__main__:extract data from url
INFO:__main__:extract data from file
INFO:__main__:get stream
INFO:__main__:transform data to windowing data
INFO:__main__:saving stream to file for max 100 secs
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:write batch to file
INFO:__main__:extract data from file
INFO:__main__:transform data to top 10 per window


,start,end,Source airport,count,window_rank
0,2021-05-05 14:37:00,2021-05-05 14:37:10,DFW,184,1
1,2021-05-05 14:37:00,2021-05-05 14:37:10,CLT,135,2
2,2021-05-05 14:37:00,2021-05-05 14:37:10,MIA,127,3
3,2021-05-05 14:37:00,2021-05-05 14:37:10,ORD,126,4
4,2021-05-05 14:37:00,2021-05-05 14:37:10,PHL,119,5
...,...,...,...,...,...
75,2021-05-05 14:38:10,2021-05-05 14:38:20,SYD,17,6
76,2021-05-05 14:38:10,2021-05-05 14:38:20,NNG,16,7
77,2021-05-05 14:38:10,2021-05-05 14:38:20,PVG,16,8
78,2021-05-05 14:38:10,2021-05-05 14:38:20,WUX,16,9


# Task 4

*Productionize your code by adding unit tests.*

I added some simple unit test to cover the code. In the future this could be expanded to increase the code coverage. Also these relativly simple unit test could be expanded with more advanced assertions based on the dataframe equality.

## Results Task 4

In [ ]:
def test_transform_data():
  """
  test function to test the tranform_data function if 10 rows are returned when > 10 rows are given
  """
  data = [("GRQ", ""),
    ("AMS", ""),
    ("EIN", ""),
    ("BCN", ""),
    ("MAD", ""),
    ("ORY", ""),
    ("TEN", ""),
    ("TNF", ""),
    ("LON", ""),
    ("FAR", ""),
    ("DUB", ""),
    ("TXL", ""),
    ("AMS", "")]

  test_schema = StructType([ \
    StructField("Source airport",StringType(), True), \
    StructField("Source", StringType(), True)])

  df = spark.createDataFrame(data=data, schema=test_schema)
  actual_df = transform_data(df)
  print(actual_df.count())

  assert actual_df.count() == 10

In [ ]:
def test_extract_data_from_url():
  test_file_url = "https://drive.google.com/uc?export=download&id=10Yfik6C8Eiyw9-flMbXAOiXwgLQPfc3w"

  test_schema = StructType([ \
    StructField("header1",StringType(), True), \
    StructField("header2", StringType(), True)])

  data = [("Schiphol", "Data"),
      ("Eindhoven Airport", "Logistiek")]

  df_expected = spark.createDataFrame(data=data, schema=test_schema)

  df_actual = extract_data_from_url(test_file_url, header=True, schema=test_schema)

  assert df_actual.count() == 2
  assert_pyspark_df_equal(df_actual, df_expected)

In [ ]:
test_transform_data()
test_extract_data_from_url()

INFO:__main__:transform data


10


INFO:__main__:extract data from url
INFO:__main__:extract data from file
